# Auction algorithm as regularized OT

The goal is to find a regularization of the OT problem such that the auction algorithm coincides with the optimization of the dual of this OT problem. The point of this notebook is to illustrate the auction algorithm and understand how it optimizes the dual of the matching problem.

## Auction algorithm implementation

In [104]:
from numba import njit
import numpy as np


@njit
def argnmax(a: np.ndarray):
    i1, i2 = -1, -1
    v1, v2 = -np.inf, -np.inf

    for i in range(len(a)):
        if a[i] > v1:
            v2 = v1
            i2 = i1

            v1 = a[i]
            i1 = i

    return i1, i2

# @njit
def auction(A: np.ndarray):
    N = len(A)
    p = np.zeros(N)
    eps = 1 / N / 1e4

    assignment = dict()         # obj to person
    unassigned = set(range(N))  # unassigned people

    while unassigned:
        i = unassigned.pop()

        m = A[i] - p
        v, w = argnmax(m)
        p[v] = p[v] + eps + m[v] - m[w]

        if v in assignment:
            unassigned.add(int(assignment.pop(v)))

        assignment[v] = i

    # numba doesn't support comprehensions
    # rev = {}
    # for k, v in assignment.items():
    #     rev[v] = k

    # return rev
    
    return p, np.max(A - p, axis=1)

In [105]:
A = np.asarray([
    [1, 2, 2],
    [2, 1, 0],
    [1, 3, 1],
])

auction(A)

(array([2.00003333, 4.00006667, 3.00006667]),
 array([-1.00003333e+00, -3.33333333e-05, -1.00003333e+00]))

## Regularized OT

$F(x) = x \ln x + (1 - x) \ln (1 - x)$

In [119]:
import cvxpy as cp
import numpy as np


def rot(C, sig):
    n, m = C.shape

    u = cp.Variable((n,))
    v = cp.Variable((m,))

    constraints = []
    
    # sum(u) + sum(v) + sig * log(1 + exp(pi - u - v))
    objective = cp.Minimize(
        cp.sum(u) + cp.sum(v)
        + sig * cp.sum(cp.logistic((C - u[:, None] - v[None, :]) / sig))
    )

    problem = cp.Problem(objective, constraints)
    problem.solve()

    return u.value, v.value

In [122]:
sig = 1e-1

u, v = rot(A, sig)
print(u, v)
1 / (1 + np.exp(- (A - u[:, None] - v[None, :]) / sig))

[1.08404717 0.43953953 1.08404717] [0.77287287 1.4173805  0.4173805 ]


array([[1.89828368e-04, 6.59860450e-03, 9.93211566e-01],
       [9.99620348e-01, 1.89828388e-04, 1.89828385e-04],
       [1.89828366e-04, 9.93211566e-01, 6.59860431e-03]])